In [1]:
import os
import numpy as np

In [2]:
def calc_weighted_B2(lattice, wave_vector):
    sum_lattice = 0
    
    for i in range(lattice.shape[0]):
        for j in range(lattice.shape[1]):
            dot = wave_vector[0]*i + wave_vector[1]*j
            sum_lattice += lattice[i,j]*np.exp(1j*dot)
    
    return sum_lattice

In [3]:
def calc_energy(lattice, bonds, cc):
    nx = lattice.shape[0]
    ny = lattice.shape[1]

    energy = 0 

    for i in range(lattice.shape[0]):
        for j in range(lattice.shape[1]):
            inn = i-1 if i-1 >= 0 else lattice.shape[0]-1
            jnn = j-1 if j-1 >= 0 else lattice.shape[1]-1

            i_down = nx + i - 1 if i-1>= 0 else nx*2-1
            j_side = jnn

            energy += cc*lattice[i,j]*lattice[inn,j]*bonds[i_down,j] + cc*lattice[i,j]*lattice[i,jnn]*bonds[i,j_side]

    return -energy

### Energy

In [7]:
cc = [1,2,3]

nis = 2
num_lattices = 3
nie = 3

L= 12

wave_vector = np.array((2*np.pi/L,0))

In [8]:
energy = np.zeros((3,2))
sum_0 = np.zeros((3,2))
sum_k = np.zeros((3,2),dtype=np.complex_)
weighted_energies_0 = np.zeros((num_lattices, nis))
weighted_energies_k = np.zeros((num_lattices, nis))

In [9]:
final_result_0 = np.zeros((num_lattices, nie))
final_result_k = np.zeros((num_lattices, nie))

In [10]:
for n in range(nie):
    for s in range(num_lattices):
        for e in range(nis):
            bonds = np.loadtxt(f"bonds/final_bonds_{n}_{s}.txt")
            lattice = np.loadtxt(f"lattices/{e}/final_lattice_{n}_{s}.txt") 

            energy[s,e] = calc_energy(lattice, bonds, cc[s])
            sum_0[s,e] = np.sum(lattice)
            sum_k[s,e] = calc_weighted_B2(lattice, wave_vector)
    
    for s in range(num_lattices):
        for e in range(nis):
            sum_0[s,e] = abs(sum_0[s,e])**2
            sum_k[s,e] = abs(sum_k[s,e])**2
            energy[s,e] = np.exp(-cc[s]*energy[s,e]/L**2)
            
    partition_function = np.sum(energy, axis=1)
    
    for s in range(num_lattices):
        for e in range(nis):
            weighted_energies_0[s,e] = energy[s,e]*sum_0[s,e]/partition_function[s]
            weighted_energies_k[s,e] = energy[s,e]*sum_k[s,e]/partition_function[s]
    
    final_result_0[n,:] = np.sum(weighted_energies_0, axis=1)
    final_result_k[n,:] = np.sum(weighted_energies_k, axis=1)

/tmp/ipykernel_260268/1348851089.py:22: ComplexWarning: Casting complex values to real discards the imaginary part
  weighted_energies_k[s,e] = energy[s,e]*sum_k[s,e]/partition_function[s]


In [12]:
print(np.sum(final_result_0,axis=0))
print(np.sum(final_result_k,axis=0))

[368.21799592 406.34125924 201.17617146]
[911.95489356 344.73464696 491.3074685 ]


In [15]:
np.sqrt(406.34125924/344.73464696-1)*(1/(2*np.sin(np.pi/L)))/L

0.0680555776834362

array([248.2820654 , 168.11434861,  10.74611801])

### B2

In [2]:
os.chdir('lattices')

In [4]:
nis = 2
num_lattices = 3

L = 12

In [5]:
wave_vector = np.array((2*np.pi/L,0))

In [6]:
for i in range(nis):
    for j in range(num_lattices):
        lattice = np.loadtxt(f"{str(i)}/final_lattice_{str(j)}.txt")
        print(calc_weighted_B2(lattice, wave_vector))

(-1.8038475772933662+11.660254037844389j)
(7.732050807568874+2.4641016151377535j)
(4.732050807568874-8.196152422706636j)
(7.464101615137753-14.928203230275514j)
(-5.196152422706634+3.000000000000001j)
(-0.5358983848622483-6.000000000000002j)


In [7]:
os.listdir('0')

['final_lattice_2.txt', 'final_lattice_0.txt', 'final_lattice_1.txt']